# LangChain 核心模块学习：Chains

对于简单的大模型应用，单独使用语言模型（LLMs）是可以的。

**但更复杂的大模型应用需要将 `LLMs` 和 `Chat Models` 链接在一起 - 要么彼此链接，要么与其他组件链接。**

LangChain 为这种“链式”应用程序提供了 `Chain` 接口。

LangChain 以通用方式定义了 `Chain`，它是对组件进行调用序列的集合，其中可以包含其他链。

In [14]:
! pip install -U langchain

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ---------------------------------------- 0.0/990.0 kB ? eta -:--:--
     ---------------------------------------- 10.2/990.0 kB ? eta -:--:--
     -- ---------------------------------- 61.4/990.0 kB 656.4 kB/s eta 0:00:02
     -- ---------------------------------- 61.4/990.0 kB 656.4 kB/s eta 0:00:02
     --- --------------------------------- 92.2/990.0 kB 525.1 kB/s eta 0:00:02
     ----- ------------------------------ 153.6/990.0 kB 657.6 kB/s eta 0:00:02
     ------- ---------------------------- 204.8/990.0 kB 778.2 kB/s eta 0:00:02
     --------- -------------------------- 256.0/990.0 kB 830.3 kB/s eta 0:00:01
     ---------------- --------------------- 440.3/990.0 kB 1.3 MB/s eta 0:00:01
     -------------------- ----------------- 532.5/990.0 kB 1.3 MB/s eta 0:00:01
     -------------------- ----------------- 532.5/990.0 kB 1.3 MB/s eta 0:00:01
     -------------------- ----------------- 532.5/990.0 kB 1.3 MB/s eta 

## LLMChain

LLMChain 是 LangChain 中最简单的链，作为其他复杂 Chains 和 Agents 的内部调用，被广泛应用。

一个LLMChain由PromptTemplate和语言模型（LLM or Chat Model）组成。它使用直接传入（或 memory 提供）的 key-value 来规范化生成 Prompt Template（提示模板），并将生成的 prompt （格式化后的字符串）传递给大模型，并返回大模型输出。

![](../images/llm_chain.png)

## Router Chain: 实现条件判断的大模型调用


这段代码构建了一个可定制的链路系统，用户可以提供不同的输入提示，并根据这些提示获取适当的响应。

主要逻辑：从`prompt_infos`创建多个`LLMChain`对象，并将它们保存在一个字典中，然后创建一个默认的`ConversationChain`，最后创建一个带有路由功能的`MultiPromptChain`。

![](../images/router_chain.png)

In [15]:
from langchain.chains.router import MultiPromptChain
from langchain_openai import OpenAI
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate

In [24]:
physics_template = """你是一位非常聪明的物理教授。
你擅长以简洁易懂的方式回答关于物理的问题。
当你不知道某个问题的答案时，你会坦诚承认。

这是一个问题：
{input}"""


math_template = """你是一位很棒的数学家。你擅长回答数学问题。
之所以如此出色，是因为你能够将难题分解成各个组成部分，
先回答这些组成部分，然后再将它们整合起来回答更广泛的问题。

这是一个问题：
{input}"""

biology_template = """你是一位非常聪明的生物老师。
你擅长以简洁易懂的方式回答关于生物学的问题。
当你不知道某个问题的答案时，你会坦诚承认。

这是一个问题：
{input}"""

computer_template = """你是一位非常出色的计算机老师。
你擅长解决计算机科学中的各种问题。
你能够将复杂的概念分解成易于理解的部分，然后逐一解答。

这是一个问题：
{input}"""

cll_template = """你是一位博学的汉语言文学老师。
你擅长解释汉语言文学中的各种问题。
你能够将复杂的文学概念和语言知识以简洁明了的方式呈现。

这是一个问题：
{input}"""

In [25]:
prompt_infos = [
    {
        "name": "物理",
        "description": "适用于回答物理问题",
        "prompt_template": physics_template,
    },
    {
        "name": "数学",
        "description": "适用于回答数学问题",
        "prompt_template": math_template,
    },
    {
        "name": "生物",
        "description": "适用于回答生物问题",
        "prompt_template": biology_template,
    },
    {
        "name": "计算机",
        "description": "适用于回答计算机问题",
        "prompt_template": computer_template,
    },
    {
        "name": "汉语言文学",
        "description": "适用于回答汉语言文学问题",
        "prompt_template": cll_template,
    },
]

In [26]:
llm = OpenAI(model_name="gpt-3.5-turbo-instruct")

In [27]:
# 创建一个空的目标链字典，用于存放根据prompt_infos生成的LLMChain。
destination_chains = {}

# 遍历prompt_infos列表，为每个信息创建一个LLMChain。
for p_info in prompt_infos:
    name = p_info["name"]  # 提取名称
    prompt_template = p_info["prompt_template"]  # 提取模板
    # 创建PromptTemplate对象
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    # 使用上述模板和llm对象创建LLMChain对象
    chain = LLMChain(llm=llm, prompt=prompt)
    # 将新创建的chain对象添加到destination_chains字典中
    destination_chains[name] = chain

# 创建一个默认的ConversationChain
default_chain = ConversationChain(llm=llm, output_key="text")

In [28]:
type(default_chain)

langchain.chains.conversation.base.ConversationChain

### 使用 LLMRouterChain 实现条件判断调用

这段代码定义了一个chain对象（LLMRouterChain），该对象首先使用router_chain来决定哪个destination_chain应该被执行，如果没有合适的目标链，则默认使用default_chain。

In [29]:
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

In [30]:
# 从prompt_infos中提取目标信息并将其转化为字符串列表
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
# 使用join方法将列表转化为字符串，每个元素之间用换行符分隔
destinations_str = "\n".join(destinations)
# 根据MULTI_PROMPT_ROUTER_TEMPLATE格式化字符串和destinations_str创建路由模板
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
# 创建路由的PromptTemplate
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
# 使用上述路由模板和llm对象创建LLMRouterChain对象
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [31]:
print(destinations)

['物理: 适用于回答物理问题', '数学: 适用于回答数学问题', '生物: 适用于回答生物问题', '计算机: 适用于回答计算机问题', '汉语言文学: 适用于回答汉语言文学问题']


In [32]:
print(destinations_str)

物理: 适用于回答物理问题
数学: 适用于回答数学问题
生物: 适用于回答生物问题
计算机: 适用于回答计算机问题
汉语言文学: 适用于回答汉语言文学问题


In [33]:
print(MULTI_PROMPT_ROUTER_TEMPLATE)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (must include ```json at the start of the respon

In [34]:
print(router_template)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
物理: 适用于回答物理问题
数学: 适用于回答数学问题
生物: 适用于回答生物问题
计算机: 适用于回答计算机问题
汉语言文学: 适用于回答汉语言文学问题

<< INPUT >>
{input}

<

In [35]:
# 创建MultiPromptChain对象，其中包含了路由链，目标链和默认链。
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

In [22]:
print(chain.invoke("黑体辐射是什么？?"))

{'input': '黑体辐射是什么？?', 'text': '\n\n黑体辐射是指理想的物体在一定温度下发出的电磁辐射，其特点是辐射功率与波长和温度相关，且在任何波长下都有辐射。它的研究对于理解物质的微观结构和能量转换有着重要意义。'}


In [23]:
print(
    chain.invoke(
        "大于40的第一个质数是多少，使得这个质数加一能被3整除？"
    )
)

{'input': '大于40的第一个质数是多少，使得这个质数加一能被3整除？', 'text': '\n\n首先，我们需要了解什么是质数。质数是指除了1和它本身之外，没有其他因数的自然数。\n所以，我们需要从大于40的数开始逐个判断是否为质数。首先我们可以跳过所有的偶数，因为除了2之外，所有的偶数都不是质数。\n那么我们可以从41开始，41是一个质数。接下来，我们需要将这个质数加一，得到42。42除以3，余数是0，说明42加一能被3整除。\n所以，大于40的第一个质数是41，使得这个质数加一能被3整除。'}


In [36]:
router_chain.verbose = True

In [37]:
print(chain.invoke("黑洞是什么？"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
物理: {'input': '什么是黑洞？'}
> Finished chain.
{'input': '什么是黑洞？', 'text': '\n\n黑洞是宇宙中一种极为密集的天体，它的密度非常高，甚至使光线无法逃离它的引力。它的形成通常是由于恒星在死亡过程中塌缩形成的，也可以是由于两个天体的碰撞造成的。黑洞有着非常强大的引力，甚至可以吞噬周围的物质，因此被称为宇宙中最强大的吸力。它的存在可以解释许多宇宙现象，如星系中心的超大质量黑洞和宇宙辐射背景。尽管我们无法直接观测黑洞，但通过间接的观测和理论模型，我们可以推断出它们的存在。 '}


In [38]:
print(chain.invoke("c++和java的区别？"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
计算机: {'input': 'c++和java的区别?'}
> Finished chain.
{'input': 'c++和java的区别?', 'text': '\n\nc++和java是两种不同的编程语言，它们有以下几点区别：\n\n1. 语言类型：c++是一种编译型语言，需要先将代码编译成可执行文件才能运行，而java是一种解释型语言，代码是通过解释器运行的。\n\n2. 内存管理：c++需要程序员手动管理内存，包括分配和释放，而java由垃圾回收机制自动管理内存，减少了程序员的工作量。\n\n3. 平台兼容性：c++是一个跨平台语言，可以在不同的操作系统上运行，但是需要根据不同的平台重新编译；而java是一种完全跨平台的语言，一次编写，到处运行。\n\n4. 面向对象：c++和java都是面向对象的语言，但是c++对面向对象的支持更加灵活，可以使用多重'}


In [42]:
print(chain.invoke("人体的细胞是如何分裂的？"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
生物: {'input': '人体细胞是如何分裂的？'}
> Finished chain.
{'input': '人体细胞是如何分裂的？', 'text': '\n\n人体细胞的分裂过程被称为有丝分裂，它是一种复杂的过程，分为前期、中期、后期和末期四个阶段。在前期，细胞的染色体会复制，变成一对同样的染色体。在中期，染色体会在细胞中排列成为同一个平面。在后期，染色体会被分开，移向细胞的两端。最后，在末期，细胞会分裂成两个新的细胞，每个细胞都包含一套完整的染色体。这样就实现了细胞的分裂和复制。'}


In [41]:
print(chain.invoke("红楼梦的作者是谁？"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
汉语言文学: {'input': '红楼梦的作者是谁？'}
> Finished chain.
{'input': '红楼梦的作者是谁？', 'text': '\n\n红楼梦的作者是曹雪芹。'}


### Homework

#### 扩展 Demo：实现生物、计算机和汉语言文学老师 PromptTemplates 及对应 Chains